In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def calculate_figsize(num_individuals):
    base_height_per_row = 0.4  
    height = max(6, min(20, num_individuals * base_height_per_row + 2))
    width = 6  
    return (width, height)

def plot_errorbar_chart_optimized(csv_file, female_max_rAMELY, male_min_rAMELY, output_plot=None, dpi=300, use_seaborn_style=True):
    """
    Optimized version using the best of both libraries.
    
    Args:
        csv_file (str): Path to CSV file
        output_plot (str): Output plot path
        figsize (tuple): Figure size
        dpi (int): Resolution
        use_seaborn_style (bool): Whether to use seaborn styling
    """
    # Read data
    df = pd.read_csv(csv_file)

    # Check required columns
    required_columns = ['Individual', 'rAMELY_mean', 'rAMELY_CI95_bottom', 'rAMELY_CI95_up', 'Sex']
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"Missing required columns: {missing_columns}")
    
    figsize = calculate_figsize(len(df))

    # Set style if requested
    if use_seaborn_style:
        sns.set_style("whitegrid")
    
    # Create figure
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    
    # Get unique sexes for coloring
    unique_sexes = df['Sex'].unique()
    colors = plt.cm.Set1(range(len(unique_sexes)))
    color_map = dict(zip(unique_sexes, colors))

    # Plot each point with error bars
    for i, (idx, row) in enumerate(df.iterrows()):
        ax.errorbar(
            x=row['rAMELY_mean'],
            y=i,
            xerr=[[row['rAMELY_mean'] - row['rAMELY_CI95_bottom']], [row['rAMELY_CI95_up'] - row['rAMELY_mean']]],
            fmt='o',
            color=color_map[row['Sex']],
            markersize=8,
            capsize=5,
            label=row['Sex'] if row['Sex'] not in df.iloc[:i]['Sex'].values else ""
        )
    
    # Add vertical dashed lines
    ax.axvline(x=female_max_rAMELY, color=color_map['F'], linestyle='--', alpha=0.7, linewidth=1.5)
    ax.axvline(x=male_min_rAMELY, color=color_map['M'], linestyle='--', alpha=0.7, linewidth=1.5)

    ax.set_ylim(-0.5, len(df) - 0.5)  # 这会消除多余的空白

    # Get limit of axes
    x_min, x_max = ax.get_xlim()
    _y_min, y_max = ax.get_ylim()
    
    # Add tag
    ax.text((x_min + female_max_rAMELY)/2, y_max * 1.001, 'XX(Female)', 
        ha='center', va='bottom', fontsize=12)
    ax.text((male_min_rAMELY + x_max)/2, y_max * 1.001, 'XY(Male)', 
        ha='center', va='bottom', fontsize=12)

    # Customize plot
    #ax.set_ylabel('Individual', fontsize=12, fontweight='bold')
    ax.set_xlabel('rAMELY', fontsize=12, fontweight='bold')
    #ax.set_title('rAMELY Value in Individuals of Known Sex', fontsize=14, fontweight='bold', y=1.05, pad=20) 
    ax.set_yticks(range(len(df)))
    ax.set_yticklabels(df['Individual'])
    
    # Add legend
    #handles, labels = ax.get_legend_handles_labels()
    #by_label = dict(zip(labels, handles))
    #ax.legend(by_label.values(), by_label.keys(), title='Sex')
    
    # Add grid
    ax.grid(None)
    
    plt.tight_layout()
    
    if output_plot:
        plt.savefig(output_plot, bbox_inches='tight', dpi=dpi)
        print(f"Plot saved to: {output_plot}")
    else:
        plt.show()
    
    plt.close()

In [44]:
plot_errorbar_chart_optimized("/mnt/data3/fan_bai/XY_protein/script/nextflow/qz_output/All_summary_statistics.test.csv",
							  0.1, 0.45,
							  "/mnt/data3/fan_bai/XY_protein/script/nextflow/qz_output/All_summary_statistics.test.pdf")

Plot saved to: /mnt/data3/fan_bai/XY_protein/script/nextflow/qz_output/All_summary_statistics.test.pdf
